In [1]:
import pandas as pd
import tensorflow as tf
from  tensorflow.contrib.learn.python.learn.estimators.dnn  import DNNClassifier
from tensorflow.contrib.layers import real_valued_column

In [99]:
apple = pd.read_csv('merged_df.csv')
#googl = pd.read_csv('WIKI_PRICES_GOOGL.csv')

In [100]:
import numpy as np

In [101]:
apple.head()

,Unnamed: 0,date,sentiment,Count,ticker,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume,diff,bin_sentiment,bin_diff
0,0,2014-05-05,53,133,AAPL,590.14,601.0000,590.00,600.9600,10252400.0,0.00,1.0,79.229879,80.687900,79.211083,80.682530,71766800.0,10.8200,1,1
1,1,2014-05-06,15,177,AAPL,601.80,604.4099,594.41,594.4100,13377300.0,0.00,1.0,80.795305,81.145700,79.803152,79.803152,93641100.0,-7.3900,1,-1
2,2,2014-05-07,40,134,AAPL,595.25,597.2900,587.73,592.3300,10102300.0,0.00,1.0,79.915927,80.189810,78.906322,79.523900,70716100.0,-2.9200,1,-1
3,3,2014-05-08,31,109,AAPL,588.25,594.4100,586.40,587.9900,8224900.0,3.29,1.0,79.418033,80.249678,79.168269,79.382931,57574300.0,-0.2600,1,-1
4,4,2014-05-09,3,67,AAPL,584.54,586.2500,580.33,585.5425,10414200.0,0.00,1.0,78.917156,79.148018,78.348775,79.052500,72899400.0,1.0025,1,1


In [102]:
aapl = apple.copy().drop(apple.columns[[0,2,3, 4, 5, 6, 7, 8, 9, 10, 11, 17, 18, ]], axis = 1)
aapl.head()

,date,adj_open,adj_high,adj_low,adj_close,adj_volume,bin_diff
0,2014-05-05,79.229879,80.687900,79.211083,80.682530,71766800.0,1
1,2014-05-06,80.795305,81.145700,79.803152,79.803152,93641100.0,-1
2,2014-05-07,79.915927,80.189810,78.906322,79.523900,70716100.0,-1
3,2014-05-08,79.418033,80.249678,79.168269,79.382931,57574300.0,-1
4,2014-05-09,78.917156,79.148018,78.348775,79.052500,72899400.0,1


In [103]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression(C=1e5)

In [104]:
InputDF = aapl.copy().drop('bin_diff', axis = 1)
InputDF = InputDF.set_index('date')

In [105]:
label = pd.qcut(apple['diff'], 2,labels=range(2))

In [112]:
label.values

[1, 0, 0, 0, 1, ..., 0, 0, 1, 0, 1]
Length: 779
Categories (2, int64): [0 < 1]

In [106]:
InputDF.head()
print (InputDF.shape)

(779, 5)


In [107]:
InputDF = InputDF.apply(lambda x:(x -x.mean())/x.std())

In [108]:
InputDF.head()

,adj_open,adj_high,adj_low,adj_close,adj_volume
date,,,,,
2014-05-05,-1.923325,-1.890850,-1.866902,-1.833370,1.255323
2014-05-06,-1.825542,-1.862243,-1.829929,-1.888248,2.270155
2014-05-07,-1.880472,-1.921975,-1.885933,-1.905676,1.206577
2014-05-08,-1.911572,-1.918234,-1.869576,-1.914473,0.596879
2014-05-09,-1.942859,-1.987076,-1.920750,-1.935094,1.307868


In [109]:
test_size = 600
xtrain, xtest = InputDF.iloc[:test_size, :], InputDF.iloc[test_size:, :]
ytrain, ytest = label[:test_size], label[test_size:]

In [110]:
t = (InputDF[:-test_size].values,label[:-test_size].values)
v = (InputDF[-test_size:].values,label[-test_size:].values)

t[0].shape

(179, 5)

In [90]:
res = logreg.fit(xtrain, ytrain)

In [91]:
from sklearn.metrics import classification_report,confusion_matrix

print (classification_report(ytest , res.predict(xtest)))
print (confusion_matrix(ytest, res.predict(xtest)))

             precision    recall  f1-score   support

          0       1.00      0.99      0.99        83
          1       0.99      1.00      0.99        96

avg / total       0.99      0.99      0.99       179

[[82  1]
 [ 0 96]]


## Training basic feed forward neural network

In [92]:
num_features = len(InputDF.columns)
dropout=0.2
hidden_1_size = 100
hidden_2_size = 25
num_classes = label.nunique()
NUM_EPOCHS=10
BATCH_SIZE=50
lr=0.0001

In [93]:
# train = (xtrain, ytrain.values)
# val = (xtest, ytest.values)
val = (InputDF[:-test_size].values, label[:-test_size].values)
train = (InputDF[-test_size:].values, label[-test_size:].values)
NUM_TRAIN_BATCHES = int(len(train[0])/BATCH_SIZE)
NUM_VAL_BATCHES = int(len(val[1])/BATCH_SIZE)

In [94]:
print (NUM_TRAIN_BATCHES)
int(len(xtrain)/BATCH_SIZE)

12


12

In [95]:
print (NUM_VAL_BATCHES)
int(len(xtest)/BATCH_SIZE)

3


3

In [96]:
class Model():
    def __init__(self):
        global_step = tf.contrib.framework.get_or_create_global_step()
        self.input_data = tf.placeholder(dtype=tf.float32,shape=[None,num_features])
        self.target_data = tf.placeholder(dtype=tf.int32,shape=[None])
        self.dropout_prob = tf.placeholder(dtype=tf.float32,shape=[])
        with tf.variable_scope("ff"):
            droped_input = tf.nn.dropout(self.input_data,keep_prob=self.dropout_prob)
            
            layer_1 = tf.contrib.layers.fully_connected(
                num_outputs=hidden_1_size,
                inputs=droped_input,
            )
            layer_2 = tf.contrib.layers.fully_connected(
                num_outputs=hidden_2_size,
                inputs=layer_1,
            )
            self.logits = tf.contrib.layers.fully_connected(
                num_outputs=num_classes,
                activation_fn =None,
                inputs=layer_2,
            )
        with tf.variable_scope("loss"):
            
            self.losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = self.logits, 
                                                                         labels = self.target_data)
            mask = (1-tf.sign(1-self.target_data)) #Don't give credit for flat days
            mask = tf.cast(mask,tf.float32)
            self.loss = tf.reduce_sum(self.losses)
        
        with tf.name_scope("train"):
            opt = tf.train.AdamOptimizer(lr)
            gvs = opt.compute_gradients(self.loss)
            self.train_op = opt.apply_gradients(gvs, global_step=global_step)
        
        with tf.name_scope("predictions"):
            self.probs = tf.nn.softmax(self.logits)
            self.predictions = tf.argmax(self.probs, 1)
            correct_pred = tf.cast(tf.equal(self.predictions, tf.cast(self.target_data,tf.int64)),tf.float64)
            self.accuracy = tf.reduce_mean(correct_pred)

In [97]:
with tf.Graph().as_default():
    model = Model()
    input_ = train[0]
    target = train[1]
    with tf.Session() as sess:
        init = tf.initialize_all_variables()
        sess.run([init])
        epoch_loss =0
        for e in range(NUM_EPOCHS):
            if epoch_loss >0 and epoch_loss <1:
                break
            epoch_loss =0
            for batch in range(0,NUM_TRAIN_BATCHES):
                
                start = batch*BATCH_SIZE
                end = start + BATCH_SIZE 
                feed = {
                    model.input_data:input_[start:end],
                    model.target_data:target[start:end],
                    model.dropout_prob:0.9
                            }
                
                _,loss,acc = sess.run(
                    [
                        model.train_op,
                        model.loss,
                        model.accuracy,
                    ]
                    ,feed_dict=feed
                )
                epoch_loss+=loss
            print('step - {0} loss - {1} acc - {2}'.format((1+batch+NUM_TRAIN_BATCHES*e),epoch_loss,acc))
                
        
        print('done training')
        final_preds =np.array([])
        final_probs =None
        for batch in range(0,NUM_VAL_BATCHES):
            
                start = batch*BATCH_SIZE
                end = start + BATCH_SIZE 
                feed = {
                    model.input_data:val[0][start:end],
                    model.target_data:val[1][start:end],
                    model.dropout_prob:1
                            }
                
                acc,preds,probs = sess.run(
                    [
                        model.accuracy,
                        model.predictions,
                        model.probs
                    ]
                    ,feed_dict=feed
                )
                print(acc)
                final_preds = np.concatenate((final_preds,preds),axis=0)
                if final_probs is None:
                    final_probs = probs
                else:
                    final_probs = np.concatenate((final_probs,probs),axis=0)
        prediction_conf = final_probs[np.argmax(final_probs, 1)]

Instructions for updating:
Use `tf.global_variables_initializer` instead.
step - 12 loss - 418.0542907714844 acc - 0.58
step - 24 loss - 417.95225524902344 acc - 0.6
step - 36 loss - 416.98297119140625 acc - 0.6
step - 48 loss - 416.33203125 acc - 0.6
step - 60 loss - 414.8281707763672 acc - 0.64
step - 72 loss - 415.0836410522461 acc - 0.6
step - 84 loss - 414.72016525268555 acc - 0.6
step - 96 loss - 415.8470573425293 acc - 0.6
step - 108 loss - 414.93212127685547 acc - 0.6
step - 120 loss - 413.87714767456055 acc - 0.6
done training
0.48
0.52
0.5


In [98]:
print (classification_report(label[:-test_size].values, final_preds))
#ytest.shape
print (final_preds.shape)

ValueError: Found input variables with inconsistent numbers of samples: [179, 150]

# Recursive Neural Nets

In [288]:
from tensorflow.contrib.layers.python.layers.initializers import xavier_initializer
from tensorflow.contrib import rnn

RNN_HIDDEN_SIZE=100
FIRST_LAYER_SIZE=1000
SECOND_LAYER_SIZE=250
NUM_LAYERS=2
BATCH_SIZE=50
NUM_EPOCHS=10
lr=0.0003
NUM_TRAIN_BATCHES = int(len(train[0])/BATCH_SIZE)
NUM_VAL_BATCHES = int(len(val[1])/BATCH_SIZE)
ATTN_LENGTH=30
beta=0

In [289]:
class RNNModel():
    def __init__(self):
        global_step = tf.contrib.framework.get_or_create_global_step()
        self.input_data = tf.placeholder(dtype=tf.float32,shape=[BATCH_SIZE,num_features])
        self.target_data = tf.placeholder(dtype=tf.int32,shape=[BATCH_SIZE])
        self.dropout_prob = tf.placeholder(dtype=tf.float32,shape=[])
        
        def makeGRUCells():
            base_cell = rnn.GRUCell(num_units=RNN_HIDDEN_SIZE,) 
            layered_cell = rnn.MultiRNNCell([base_cell] * NUM_LAYERS,state_is_tuple=False) 
            attn_cell =tf.contrib.rnn.AttentionCellWrapper(cell=layered_cell,attn_length=ATTN_LENGTH,state_is_tuple=False)
            return attn_cell
        
        self.gru_cell = makeGRUCells()
        self.zero_state = self.gru_cell.zero_state(1, tf.float32)
        
        self.start_state = tf.placeholder(dtype=tf.float32,shape=[1,self.gru_cell.state_size])
        
        

        with tf.variable_scope("ff",initializer=xavier_initializer(uniform=False)):
            droped_input = tf.nn.dropout(self.input_data,keep_prob=self.dropout_prob)
            
            layer_1 = tf.contrib.layers.fully_connected(
                num_outputs=FIRST_LAYER_SIZE,
                inputs=droped_input,
                
            )
            layer_2 = tf.contrib.layers.fully_connected(
                num_outputs=RNN_HIDDEN_SIZE,
                inputs=layer_1,
                
            )
            
        
        split_inputs = tf.reshape(droped_input,shape=[1,BATCH_SIZE,num_features],name="reshape_l1") # Each item in the batch is a time step, iterate through them
        split_inputs = tf.unstack(split_inputs,axis=1,name="unpack_l1")
        states =[]
        outputs =[]
        with tf.variable_scope("rnn",initializer=xavier_initializer(uniform=False)) as scope:
            state = self.start_state
            for i, inp in enumerate(split_inputs):
                if i >0:
                    scope.reuse_variables()
                
                output, state = self.gru_cell(inp, state)
                states.append(state)
                outputs.append(output)
        self.end_state = states[-1]
        outputs = tf.stack(outputs,axis=1) # Pack them back into a single tensor
        outputs = tf.reshape(outputs,shape=[BATCH_SIZE,RNN_HIDDEN_SIZE])
        self.logits = tf.contrib.layers.fully_connected(
            num_outputs=num_classes,
            inputs=outputs,
            activation_fn=None
        )

            
        with tf.variable_scope("loss"):
            self.penalties =    tf.reduce_sum([beta*tf.nn.l2_loss(var) for var in tf.trainable_variables()])

            
            self.losses = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = self.logits,
                                                                         labels = self.target_data)
            self.loss = tf.reduce_sum(self.losses + beta*self.penalties)
        
        with tf.name_scope("train_step"):
            opt = tf.train.AdamOptimizer(lr)
            gvs = opt.compute_gradients(self.loss)
            self.train_op = opt.apply_gradients(gvs, global_step=global_step)
        
        with tf.name_scope("predictions"):
            probs = tf.nn.softmax(self.logits)
            self.predictions = tf.argmax(probs, 1)
            correct_pred = tf.cast(tf.equal(self.predictions, tf.cast(self.target_data,tf.int64)),tf.float64)
            self.accuracy = tf.reduce_mean(correct_pred)

# training the RNN

In [290]:
with tf.Graph().as_default():
    model = RNNModel()
    input_ = train[0]
    target = train[1]
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run([init])
        loss = 2000
        
        for e in range(NUM_EPOCHS):
            state = sess.run(model.zero_state)
            epoch_loss =0
            for batch in range(0,NUM_TRAIN_BATCHES):
                start = batch*BATCH_SIZE
                end = start + BATCH_SIZE 
                feed = {
                    model.input_data:input_[start:end],
                    model.target_data:target[start:end],
                    model.dropout_prob:0.5,
                    model.start_state:state
                            }
                _,loss,acc,state = sess.run(
                    [
                        model.train_op,
                        model.loss,
                        model.accuracy,
                        model.end_state
                    ]
                    ,feed_dict=feed
                )
                epoch_loss+=loss
                
            print('step - {0} loss - {1} acc - {2}'.format((e),epoch_loss,acc))
        final_preds =np.array([])
        for batch in range(0,NUM_VAL_BATCHES):
                start = batch*BATCH_SIZE
                end = start + BATCH_SIZE 
                feed = {
                    model.input_data:val[0][start:end],
                    model.target_data:val[1][start:end],
                    model.dropout_prob:1,
                    model.start_state:state
                            }
                acc,preds,state = sess.run(
                    [
                        model.accuracy,
                        model.predictions,
                        model.end_state
                    ]
                    ,feed_dict=feed
                )
                print(acc)
                assert len(preds) == BATCH_SIZE
                final_preds = np.concatenate((final_preds,preds),axis=0)

step - 0 loss - 423.2655487060547 acc - 0.54
step - 1 loss - 424.90478515625 acc - 0.54
step - 2 loss - 410.9787902832031 acc - 0.58
step - 3 loss - 411.7039451599121 acc - 0.54
step - 4 loss - 410.9993782043457 acc - 0.58
step - 5 loss - 409.0864067077637 acc - 0.54
step - 6 loss - 408.6769714355469 acc - 0.56
step - 7 loss - 408.57334899902344 acc - 0.54
step - 8 loss - 409.7863540649414 acc - 0.54
step - 9 loss - 411.15460205078125 acc - 0.54
0.54
0.4
0.56


In [291]:
print (classification_report(ytest, final_preds))

ValueError: Found input variables with inconsistent numbers of samples: [179, 150]